In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from glob import glob
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
obdir = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/observations/'

### Choose which sites to train the SVR model on
Can be all available, or a defined list 'site_list' passed, or single station as list e.g. ['CLN']<br>
Use all with an exclude list is another alternative

In [3]:
flist = glob(obdir + 'combined/*.pd')

# This can be a manual site list if desired
site_list = np.unique([f.split('/')[-1].split('_')[0] for f in flist])

# site_list = [s for s in site_list if 'BSNF' not in s]
#site_list = ['CLN', 'AGD', 'ALTA']

site_list

array(['AGD', 'ALTA', 'BCC', 'BSNFDC', 'BSNFEX', 'BSNFJE', 'CLNX', 'PVC',
       'SLB'], dtype='<U6')

### For each site, choose which files to use
For now, we will only use one set of observations from each site so samples remain independent<br>
We can change this behavior down the road to use all or some intervals<br>
Favor 'short' for the shortest interval available e.g. 6h, 'long' for longest interval e.g. 24h

In [4]:
favor = 'short' #'long'

flist = []
for site in site_list:
    
    site_files = glob(obdir + 'combined/%s*.pd'%site)
    
#     if len(site_files) > 1:
#         if favor == 'short':
#             flist.append(
#                 site_files[np.argmin([int(f.split('.')[-2].replace('h', '')) 
#                     for f in site_files])])
            
#         elif favor == 'long':
#             np.argmax([int(f.split('.')[-2].replace('h', '')) for f in site_files])
    
#     else:
    flist.append(site_files)#[0])
        
flist = np.hstack(flist)
flist

array(['/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/observations/combined/AGD_2004_2019.24h.pd',
       '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/observations/combined/AGD_2004_2019.12h.pd',
       '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/observations/combined/ALTA_1980_2020.24h.pd',
       '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/observations/combined/BCC_2004_2019.24h.pd',
       '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/observations/combined/BCC_2004_2019.12h.pd',
       '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/observations/combined/BSNFDC_2005_2019.24h.pd',
       '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/observations/combined/BSNFEX_2012_2019.24h.pd',
       '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/observations/combined/BSNFJE_2005_2019.24h.pd',
       '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/observati

### Establish which model variables and levels to use
This is entirely based on the model that the SVR will be applied to - maximize the predictor set<br>
Helpful to open a sample GFS/NAM/HRRR grid or profile to derive the variable list from

### Import the compiled data
Prune unused variables on import based on the list above<br>
Sub-surface levels have already been dealt with in the compilation script - modify methods there<br>
Note that 12 vs 24 hour intervals have also been dealt with above, if both desired... write that in

In [16]:
use_var_type = ['mean']#, 'max', 'min']

data = []
for f in flist:
    
    site = f.split('/')[-1].split('_')[0]
    interval = int(f.split('/')[-1].split('.')[-2].replace('h', ''))
    
    df = pd.read_pickle(f)
    
    include_keys = ['T', 'R', 'U', 'V', 'Z']
    keys = ['slr', 'swe_mm']
    keys.extend(np.hstack([[k for k in df.keys()
                            if ((vt in k) & (k.split('_')[0] in include_keys))] 
                           for vt in use_var_type]))
        
    df = df.loc[:, keys].rename(columns={[k for k in keys if 'swe' in k][0]:'swe_mm'})
    df = df.loc[:, :].rename(columns={[k for k in keys if 'swe' in k][0]:'swe_mm'})
    df = df.rename(columns={[k for k in keys if 'slr' in k][0]:'slr'})
    df = df.drop(columns=[k for k in keys if 'auto' in k])
    
    # df.insert(0, 'site', np.full(df.index.size, fill_value=site, dtype='U10'))
    doy = [int(pd.to_datetime(d).strftime('%j')) for d in df.index]
    #df.insert(2, 'day_of_year', doy)
    
    data.append(df.reset_index().drop(columns='time'))

data = pd.concat(data, sort=False)

# Treat the mean value as the instantaneous value for later applications,
# we can change this behavior later on if desired. 
# An alternate method would be to keep the 'mean' tag through training 
# and choose behavior upon application
data = data.rename(columns={k:k.replace('_mean', '') for k in data.keys()})

print(data.shape)
data.head()

(14340, 52)


,slr,swe_mm,T_01agl,U_01agl,V_01agl,Z_01agl,R_01agl,T_02agl,U_02agl,V_02agl,...,T_09agl,U_09agl,V_09agl,Z_09agl,R_09agl,T_10agl,U_10agl,V_10agl,Z_10agl,R_10agl
0,5.88,12.95,274.660889,-7.088934,10.160769,30329.960938,75.513489,271.041412,-3.992080,12.588694,...,232.852051,-6.405712,26.100204,92202.718750,60.803410,224.151947,-4.833603,25.590794,104153.101562,49.459042
1,5.32,11.94,264.753296,4.310259,-6.581493,29981.410156,71.208900,263.660675,5.649075,-11.672703,...,226.124588,4.332445,-28.059399,90629.929688,80.730492,216.126663,3.423474,-34.172352,102199.382812,93.531425
2,12.21,21.84,269.180389,8.406146,-2.683464,30030.525391,91.871696,266.840668,11.762459,-7.780785,...,229.437149,17.053003,-9.621124,91323.421875,79.696442,219.605240,19.694160,-9.853032,103065.382812,79.703049
3,12.20,25.40,269.066162,8.432843,-0.374978,29730.644531,91.111023,265.930908,11.585249,-3.159128,...,227.716583,22.342390,0.114759,90622.953125,84.874001,220.347107,24.896339,-1.074835,102320.210938,61.295341
4,16.34,13.21,263.492554,7.508132,-4.722282,29630.097656,68.908722,260.387848,9.471686,-6.016488,...,226.809479,36.793125,-22.943510,89165.539062,47.654980,220.883804,37.234291,-25.007586,100866.343750,41.149246


### Hard trim data based on criteria

In [1]:
min_slr, max_slr = 2.5, 30
max_T650 = 0 + 273.15
min_swe_mm = 2.54

data = data[data['slr'] >= min_slr]
data = data[data['slr'] <= max_slr]
data = data[data['T_01agl'] <= max_T650]
data = data[data['swe_mm'] >= min_swe_mm]

dkeys = [k for k in data.keys() if 'T_' in k]
dkeys.insert(0, 'slr')

data = data[dkeys]

print(data)

NameError: name 'data' is not defined

In [ ]:
fig, ax = plt.subplots(1, facecolor='w', figsize=(10, 8))

data['slr'].hist(ax=ax, color='k', width=1.8, bins=np.arange(0, 51, 2))

ax.set_xticks(np.arange(0, 51, 2))
ax.set_xlim([0, 50])
ax.grid(False)
ax.set_title('n=%d\n%s - %s'%(len(data['slr']), 1980, 2020))

stats = [np.nanmean(data['slr']), np.nanstd(data['slr']), np.percentile(data['slr'], [75, 50, 25])]
stats

ax.text(x=30, y=1000, s='Average SLR: %.1f\nStandard Dev: %.1f\n75th Percentile: %.1f\n50th Percentile: %.1f\n25th Percentile: %.1f'%(stats[0], stats[1], stats[2][0], stats[2][1], stats[2][2]), fontsize=18, weight='bold')

<hr><hr>

### Investigate basic bulk statistics and linear correlations
Quanitfy how certain variables are related and produce plots to visualize this<br>

#### *Removed: Normalize/Standardize
Moved down to where datasets are split. If need be we can normalize for the plots below here too...

#### Pair Plots

#### Correlation Matrix, R2 Matrix

#### Linear Correlation Verical Plots, R2 Vertical Plots

#### Identify autocorrelated and other dependent predictors
Mask these if desired

<hr><hr>

### Split Train/Test/Validate Sets
If we want to expand the hyperparameter tuning, we can test for sensitivity to random sample<br>
by looping from bottom of code block back to here<br>

In [ ]:
# int(slr) for stratification (> 1 ct per class label)
data = data.dropna()
fac = 5
slr = np.round(data['slr']/fac, 0)*fac
slr.value_counts()

In [ ]:
print('Total: %d'%len(data))

# Split into train/test sets
train_size, test_size, random_state = None, 0.33, 5
X_train, X_test = train_test_split(data, 
                                       test_size=test_size, train_size=train_size, 
                                       random_state=random_state, stratify=slr)

# Perform a secondary split if separate validation set required

# Split off the target variable now that TTsplit is done
y_train, y_test = X_train.pop('slr'), X_test.pop('slr')
# y_train = np.round(y_train/fac, 0)*fac

print('Train: {}\nTest: {}\nValidate: {}'.format(X_train.shape[0], X_test.shape[0], None))

train_stats = X_train.describe().T
train_stats.head(10)

### Normalize/Standardize the data
We are using z-score normalization for now but other methods exist<br>
This is an absolute must for working with any of the ML models available to us<br>
Normalize based on the TRAINING dataset

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler

In [ ]:
scaler = RobustScaler(quantile_range=(25, 75)).fit(X_train)

X_train_norm = pd.DataFrame(scaler.transform(X_train.loc[:, list(X_train.keys())]), columns=X_train.keys())
X_test_norm = pd.DataFrame(scaler.transform(X_test.loc[:, list(X_train.keys())]), columns=X_train.keys())

X_train_norm.describe().T.head(10)

Print out plots describing the variability in the data (+ before/after normalization?)<br>
It would be valuable to do a similar set of/novel comparison plots with the ERA5 vs GFS<br>
for each variable/level in the validation script (separate from and following this script)

### EOF/PCA Reduction of highly dimensional data
There are implementations of atmospheric data in ML models that heavily leverage<br>
EOFs to reduce highly dimensional data to simpler components<br>
Evaluate if this is viable here as it will be applied to raw variables later on

### Use LASSO/Ridge regression to determine predictor rank/impact
L1/L2 Parameter Tuning -- Use to select relevant predictors, reduce irrelevant to zero, rank by influence

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train_norm, y_train)

lr_train_score = lr.score(X_train_norm, y_train)
lr_test_score = lr.score(X_test_norm, y_test)

print('Linear Regression Train Score: %.3f'%lr_train_score)
print('Linear Regression Test Score: %.3f'%lr_test_score)

In [ ]:
# fig, ax = plt.subplots(1, 1, facecolor='w', figsize=(8, 14))

# ranksort = np.argsort(abs(lr.coef_))[::-1]
# lr_coefs = lr.coef_[ranksort]
# lr_keys = X_train_norm.keys()[ranksort]
# mask = lr_coefs != 0

# ax.axvline(0, color='k', linewidth=3, zorder=11)
# ax.barh(lr_keys[mask], lr_coefs[mask], color='green', zorder=10, height=0.97)

# for i, k in enumerate(lr_keys[mask]):
#     if lr_coefs[i] != 0:
#         ax.text(lr_coefs[i]/2, k, k, zorder=20)

# ax.invert_yaxis()
# ax.axes.get_yaxis().set_visible(False)
# ax.set_title('Linear Regression Coefs')
# ax.set_xlabel('Linear Coef Value')
# ax.grid(zorder=-1)



In [ ]:
test_predictions = lr.predict(X_test_norm)
lr_mae = np.nanmean(abs(test_predictions - y_test))

maxslr = test_predictions.max() if test_predictions.max() > y_test.max() else y_test.max()
maxslr += 5

fig, axs = plt.subplots(1, 3, figsize=(30, 8), facecolor='w')

ax = axs[0]
maxslr = y_test.max() if y_test.max() > y_train.max() else y_train.max()

ax.hist(y_train, bins=np.arange(0, maxslr, 2), color='g', edgecolor='k', alpha=1.0, label='Train SLR\nn=%d'%len(y_train))
ax.hist(y_test, bins=np.arange(0, maxslr, 2), color='C0', edgecolor='k', alpha=1.0, label='Test SLR\nn=%d'%len(y_test))
ax.legend()

ax.set_xticks(np.arange(0, maxslr+1, 5))
ax.set_xticklabels(np.arange(0, maxslr+1, 5).astype(int))
ax.grid()

ax = axs[1]
maxslr = test_predictions.max() if test_predictions.max() > y_test.max() else y_test.max()
maxslr += 5
ax.scatter(y_test, test_predictions, c='k', s=50, marker='+', linewidth=0.75)
ax.set_xlabel('Observed SLR')
ax.set_ylabel('Predicted SLR')
ax.plot([0, maxslr], [0, maxslr])
ax.set_xlim([0, maxslr])
ax.set_ylim([0, maxslr])
ax.set_aspect('equal')
ax.grid()
axs[1].set_title('Linear Model\nR2: %.3f\nMAE: %.3f'%(lr_test_score, lr_mae))

ax = axs[2]
error = test_predictions - y_test
ax.hist(error, bins=np.arange(-30, 30, 2), edgecolor='k')
ax.set_xlabel('Prediction Error')
ax.set_ylabel('Count')
ax.grid()

# fig.suptitle('MultiLayer Perceptron (Simple ANN) [%d Hidden Layer Sizes]'%layer_sizes)

In [ ]:
from sklearn.linear_model import Ridge

# The higher the alpha value, more restriction on the coefficients; 
# low alpha > more generalization, coefficients are barely restricted
rr_alpha_tune = []
for alpha in np.append(np.arange(0.01, 1, 0.01), np.arange(1, 1000, 1)):

    rr = Ridge(alpha=alpha)
    rr.fit(X_train, y_train)

    rr_train_score = rr.score(X_train, y_train)
    rr_test_score = rr.score(X_test, y_test)
    
    rr_alpha_tune.append([alpha, rr_train_score, rr_test_score, rr])
rr_alpha_tune = np.array(rr_alpha_tune)

alpha, rr_train_score, rr_test_score, rr = rr_alpha_tune[np.argmax(rr_alpha_tune[:, 2])]
print('Ridge Regression alpha: %.3f'%alpha)
print('Ridge Regression Train Score: %.3f'%rr_train_score)
print('Ridge Regression Test Score: %.3f'%rr_test_score)

In [ ]:
# fig, ax = plt.subplots(1, 1, facecolor='w', figsize=(8, 14))

# ranksort = np.argsort(abs(rr.coef_))[::-1]
# rr_coefs = rr.coef_[ranksort]
# rr_keys = X_train_norm.keys()[ranksort]
# mask = rr_coefs != 0

# ax.axvline(0, color='k', linewidth=3, zorder=11)
# ax.barh(rr_keys[mask], rr_coefs[mask], color='green', zorder=10, height=0.97)

# for i, k in enumerate(rr_keys[mask]):
#     if rr_coefs[i] != 0:
#         ax.text(rr_coefs[i]/2, k, k, zorder=20)

# ax.invert_yaxis()
# ax.axes.get_yaxis().set_visible(False)
# ax.set_title('Ridge Regression Coefs')
# ax.set_xlabel('Ridge Coef Value')
# ax.grid(zorder=-1)

In [ ]:
test_predictions = lr.predict(X_test_norm)
rr_mae = np.nanmean(abs(test_predictions - y_test))

maxslr = test_predictions.max() if test_predictions.max() > y_test.max() else y_test.max()
maxslr += 5

fig, axs = plt.subplots(1, 3, figsize=(30, 8), facecolor='w')

ax = axs[0]
maxslr = y_test.max() if y_test.max() > y_train.max() else y_train.max()

ax.hist(y_train, bins=np.arange(0, maxslr, 2), color='g', edgecolor='k', alpha=1.0, label='Train SLR\nn=%d'%len(y_train))
ax.hist(y_test, bins=np.arange(0, maxslr, 2), color='C0', edgecolor='k', alpha=1.0, label='Test SLR\nn=%d'%len(y_test))
ax.legend()

ax.set_xticks(np.arange(0, maxslr+1, 5))
ax.set_xticklabels(np.arange(0, maxslr+1, 5).astype(int))
ax.grid()

ax = axs[1]
maxslr = test_predictions.max() if test_predictions.max() > y_test.max() else y_test.max()
maxslr += 5
ax.scatter(y_test, test_predictions, c='k', s=50, marker='+', linewidth=0.75)
ax.set_xlabel('Observed SLR')
ax.set_ylabel('Predicted SLR')
ax.plot([0, maxslr], [0, maxslr])
ax.set_xlim([0, maxslr])
ax.set_ylim([0, maxslr])
ax.set_aspect('equal')
ax.grid()
axs[1].set_title('Linear Model\nR2: %.3f\nMAE: %.3f'%(rr_test_score, rr_mae))

ax = axs[2]
error = test_predictions - y_test
ax.hist(error, bins=np.arange(-30, 30, 2), edgecolor='k')
ax.set_xlabel('Prediction Error')
ax.set_ylabel('Count')
ax.grid()

# fig.suptitle('MultiLayer Perceptron (Simple ANN) [%d Hidden Layer Sizes]'%layer_sizes)

In [ ]:
from sklearn.linear_model import Lasso

lasso_alpha_tune = []
for alpha in np.arange(0.001, .01, 0.001):

    lasso = Lasso(alpha=alpha, max_iter=10e5)
    lasso.fit(X_train_norm, y_train)

    lasso_train_score = lasso.score(X_train_norm, y_train)
    lasso_test_score = lasso.score(X_test_norm, y_test)
    lasso_coeff_used = np.sum(lasso.coef_ != 0)
    
    lasso_alpha_tune.append([alpha, lasso_train_score, lasso_test_score, lasso])
lasso_alpha_tune = np.array(lasso_alpha_tune)

alpha, lasso_train_score, lasso_test_score, lasso = lasso_alpha_tune[np.argmax(lasso_alpha_tune[:, 2])]
print('Lasso alpha: %.3f'%alpha)
print('Lasso Train Score: %.3f'%lasso_train_score)
print('Lasso Test Score: %.3f'%lasso_test_score)
print('Number of Features Used: %d'%lasso_coeff_used)

In [ ]:
# fig, ax = plt.subplots(1, 1, facecolor='w', figsize=(8, 14))

# ranksort = np.argsort(abs(lasso.coef_))[::-1]
# lasso_coefs = lasso.coef_[ranksort]
# lasso_keys = X_train_norm.keys()[ranksort]
# mask = lasso_coefs != 0

# ax.axvline(0, color='k', linewidth=3, zorder=11)
# ax.barh(lasso_keys[mask], lasso_coefs[mask], color='green', zorder=10, height=0.97)

# for i, k in enumerate(lasso_keys[mask]):
#     if lasso_coefs[i] != 0:
#         ax.text(lasso_coefs[i]/2, k, k, zorder=20)

# ax.invert_yaxis()
# ax.axes.get_yaxis().set_visible(False)
# ax.set_title('Lasso Regression Coefs')
# ax.set_xlabel('Lasso Coef Value')
# ax.grid(zorder=-1)

# plt.show()

In [ ]:
test_predictions = lr.predict(X_test_norm)
lasso_mae = np.nanmean(abs(test_predictions - y_test))

maxslr = test_predictions.max() if test_predictions.max() > y_test.max() else y_test.max()
maxslr += 5

fig, axs = plt.subplots(1, 3, figsize=(30, 8), facecolor='w')

ax = axs[0]
maxslr = y_test.max() if y_test.max() > y_train.max() else y_train.max()

ax.hist(y_train, bins=np.arange(0, maxslr, 2), color='g', edgecolor='k', alpha=1.0, label='Train SLR\nn=%d'%len(y_train))
ax.hist(y_test, bins=np.arange(0, maxslr, 2), color='C0', edgecolor='k', alpha=1.0, label='Test SLR\nn=%d'%len(y_test))
ax.legend()

ax.set_xticks(np.arange(0, maxslr+1, 5))
ax.set_xticklabels(np.arange(0, maxslr+1, 5).astype(int))
ax.grid()

ax = axs[1]
maxslr = test_predictions.max() if test_predictions.max() > y_test.max() else y_test.max()
maxslr += 5
ax.scatter(y_test, test_predictions, c='k', s=50, marker='+', linewidth=0.75)
ax.set_xlabel('Observed SLR')
ax.set_ylabel('Predicted SLR')
ax.plot([0, maxslr], [0, maxslr])
ax.set_xlim([0, maxslr])
ax.set_ylim([0, maxslr])
ax.set_aspect('equal')
ax.grid()
axs[1].set_title('Lasso Model\nR2: %.3f\nMAE: %.3f'%(lasso_test_score, lasso_mae))

ax = axs[2]
error = test_predictions - y_test
ax.hist(error, bins=np.arange(-30, 30, 2), edgecolor='k')
ax.set_xlabel('Prediction Error')
ax.set_ylabel('Count')
ax.grid()

# fig.suptitle('MultiLayer Perceptron (Simple ANN) [%d Hidden Layer Sizes]'%layer_sizes)

In [ ]:
from sklearn.linear_model import ElasticNet

# Grid-search for best parameters? (alpha and l1_ratio)

elastic = ElasticNet(alpha=0.009, l1_ratio=0.1, max_iter=10e5)
elastic.fit(X_train_norm, y_train)

elastic_train_score = elastic.score(X_train_norm, y_train)
elastic_test_score = elastic.score(X_test_norm, y_test)
elastic_coeff_used = np.sum(elastic.coef_ != 0)
    
print('Elastic Train Score: %.3f'%elastic_train_score)
print('Elastic Test Score: %.3f'%elastic_test_score)

In [ ]:
# fig, ax = plt.subplots(1, 1, facecolor='w', figsize=(8, 14))

# ranksort = np.argsort(abs(elastic.coef_))[::-1]
# elastic_coefs = elastic.coef_[ranksort]
# elastic_keys = X_train_norm.keys()[ranksort]
# mask = elastic_coefs != 0

# ax.axvline(0, color='k', linewidth=3, zorder=11)
# ax.barh(elastic_keys[mask], elastic_coefs[mask], color='green', zorder=10, height=0.97)

# for i, k in enumerate(elastic_keys[mask]):
#     if elastic_coefs[i] != 0:
#         ax.text(elastic_coefs[i]/2, k, k, zorder=20)

# ax.invert_yaxis()
# ax.axes.get_yaxis().set_visible(False)
# ax.set_title('Elastic Regression Coefs')
# ax.set_xlabel('Elastic Coef Value')
# ax.grid(zorder=-1)

# plt.show()

In [ ]:
test_predictions = elastic.predict(X_test_norm)
elastic_mae = np.nanmean(abs(test_predictions - y_test))

maxslr = test_predictions.max() if test_predictions.max() > y_test.max() else y_test.max()
maxslr += 5

fig, axs = plt.subplots(1, 3, figsize=(30, 8), facecolor='w')

ax = axs[0]
maxslr = y_test.max() if y_test.max() > y_train.max() else y_train.max()

ax.hist(y_train, bins=np.arange(0, maxslr, 2), color='g', edgecolor='k', alpha=1.0, label='Train SLR\nn=%d'%len(y_train))
ax.hist(y_test, bins=np.arange(0, maxslr, 2), color='C0', edgecolor='k', alpha=1.0, label='Test SLR\nn=%d'%len(y_test))
ax.legend()

ax.set_xticks(np.arange(0, maxslr+1, 5))
ax.set_xticklabels(np.arange(0, maxslr+1, 5).astype(int))
ax.grid()

ax = axs[1]
maxslr = test_predictions.max() if test_predictions.max() > y_test.max() else y_test.max()
maxslr += 5
ax.scatter(y_test, test_predictions, c='k', s=50, marker='+', linewidth=0.75)
ax.set_xlabel('Observed SLR')
ax.set_ylabel('Predicted SLR')
ax.plot([0, maxslr], [0, maxslr])
ax.set_xlim([0, maxslr])
ax.set_ylim([0, maxslr])
ax.set_aspect('equal')
ax.grid()
axs[1].set_title('Elastic Model\nR2: %.3f\nMAE: %.3f'%(elastic_test_score, elastic_mae))

ax = axs[2]
error = test_predictions - y_test
ax.hist(error, bins=np.arange(-30, 30, 2), edgecolor='k')
ax.set_xlabel('Prediction Error')
ax.set_ylabel('Count')
ax.grid()

# fig.suptitle('MultiLayer Perceptron (Simple ANN) [%d Hidden Layer Sizes]'%layer_sizes)

#### If we want to use feature selection, we can do so here.
Can choose the model from which to take features, do so manually, or not at all (set equal to X_train.keys())

In [ ]:
# print('Top Predictor: %s'%elastic_keys[0])
# fig, axs = plt.subplots(3, 3, figsize=(18, 18), facecolor='w')
# axs = axs.flatten()

# for i, ax in enumerate(axs):
#     ik, ic = elastic_keys[i], elastic_coefs[i]
#     ax.scatter(X_train_norm[ik], y_train, c='k', marker='+', s=65, linewidth=0.5)
#     ax.set_title('(%d: %.2f) %s'%(i+1, ic, ik))
#     ax.grid()

# plt.show()

In [ ]:
# drop_low_import = False

# if drop_low_import:
#     feature_selection = elastic_keys[mask]
#     feature_selection

#     pre_select = X_train_norm.keys()
#     X_train_norm = X_train_norm[feature_selection]
#     X_test_norm = X_test_norm[feature_selection]
#     post_select = X_train_norm.keys()

#     print('dropped:', [k for k in pre_select if k not in post_select])

#     print(X_train_norm.shape)
#     print(X_test_norm.shape)

In [ ]:
from sklearn.neural_network import MLPRegressor

for layer_sizes in [1000]:
    mlp = MLPRegressor(
        hidden_layer_sizes=(layer_sizes,), 
        activation='relu', 
        solver='sgd', 
        alpha=0.00001, 
        batch_size='auto', 
        learning_rate='adaptive', 
        learning_rate_init=0.01, 
        power_t=0.5, 
        max_iter=50000, 
        shuffle=True, 
        random_state=random_state, 
        tol=0.0001, 
        verbose=False, 
        warm_start=False, 
        momentum=0.7, 
        nesterovs_momentum=True, 
        early_stopping=True, 
        validation_fraction=0.3, 
        beta_1=0.5, 
        beta_2=0.999, 
        epsilon=1e-8)

    mlp.fit(X_train_norm, y_train)

    mlp_train_score = mlp.score(X_train_norm, y_train)
    mlp_test_score = mlp.score(X_test_norm, y_test)
    
    # mlp_coeff_used = np.sum(mlp.coef_ != 0)

    print('Layer Sizes: %d'%layer_sizes)
    print('MLP Train Score: %.3f'%mlp_train_score)
    print('MLP Test Score: %.3f'%mlp_test_score)
    print()

In [ ]:
test_predictions = mlp.predict(X_test_norm)
mlp_mae = np.nanmean(abs(test_predictions - y_test))

maxslr = test_predictions.max() if test_predictions.max() > y_test.max() else y_test.max()
maxslr += 5

fig, axs = plt.subplots(1, 3, figsize=(30, 8), facecolor='w')

ax = axs[0]
maxslr = y_test.max() if y_test.max() > y_train.max() else y_train.max()

ax.hist(y_train, bins=np.arange(0, maxslr, 2), color='g', edgecolor='k', alpha=1.0, label='Train SLR\nn=%d'%len(y_train))
ax.hist(y_test, bins=np.arange(0, maxslr, 2), color='C0', edgecolor='k', alpha=1.0, label='Test SLR\nn=%d'%len(y_test))
ax.legend()

ax.set_xticks(np.arange(0, maxslr+1, 5))
ax.set_xticklabels(np.arange(0, maxslr+1, 5).astype(int))
ax.grid()

ax = axs[1]
maxslr = test_predictions.max() if test_predictions.max() > y_test.max() else y_test.max()
maxslr += 5
ax.scatter(y_test, test_predictions, c='k', s=50, marker='+', linewidth=0.75)
ax.set_xlabel('Observed SLR')
ax.set_ylabel('Predicted SLR')
ax.plot([0, maxslr], [0, maxslr])
ax.set_xlim([0, maxslr])
ax.set_ylim([0, maxslr])
ax.set_aspect('equal')
ax.grid()
axs[1].set_title('R2: %.3f\nMAE: %.3f'%(mlp_test_score, mlp_mae))

ax = axs[2]
error = test_predictions - y_test
ax.hist(error, bins=np.arange(-30, 30, 2), edgecolor='k')
ax.set_xlabel('Prediction Error')
ax.set_ylabel('Count')
ax.grid()

fig.suptitle('MultiLayer Perceptron (Simple ANN) [%d Hidden Layer Sizes]'%layer_sizes)

<hr><hr>

### Implement Support Vector Regression Model
(Other ML models like ANN from keras, tensorflow, pytorch may be used in this block)

Implement a K-Fold Cross Validation to assess model performance (Optional, recommended)

Make the SVR model multiprocess friendly

In [ ]:
def MARE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))

def SVR_mp(_params):
    
    print('.', end='')
    
    _i, _j, _C, _e = _params
    
    _model = SVR(
                C=_C, #Ridge regularization parameter for (L2)^2 penalty
                epsilon=_e, #Specifies the epsilon-tube within which no penalty is associated in the training loss function
                kernel='rbf', #'linear', 'polynomial', 'rbf'
                degree=3, #pass interger for 'polynomial' kernel, ignored otherwise
                tol=0.001, #stopping tolerance
                shrinking=False, 
                cache_size=200, 
                verbose=False)
    
    _model.fit(X_train_norm, y_train)
    
    test_predictions = _model.predict(X_test_norm).flatten()
    _r2 = _model.score(X_test_norm, y_test) #sklearn.metrics.r2_score(y_test.values.flatten(), test_predictions)
    _mse = sklearn.metrics.mean_squared_error(y_test.values.flatten(), test_predictions)
    _mae = sklearn.metrics.mean_absolute_error(y_test.values.flatten(), test_predictions)
    _mare = MARE(y_test.values.flatten(), test_predictions)
    
    return (_i, _j, _C, _e, _r2, _mae, _mse, _mare, _model)

In [ ]:
import multiprocessing as mp
from sklearn.svm import SVR
import sklearn

# Best test
crange = np.arange(1, 100, 1)
erange = np.arange(1.0, 5.1, .25)

# crange = np.arange(1, 25, 1)
# erange = np.arange(0.1, 3.1, .1)

params = {}
params['r2'] = np.zeros((len(crange), len(erange)))
params['mae'] = np.zeros((len(crange), len(erange)))
params['mse'] = np.zeros((len(crange), len(erange)))
params['mare'] = np.zeros((len(crange), len(erange)))
params['model'] = np.empty((len(crange), len(erange)), dtype='object')
params['epsilon'] = np.zeros((len(crange), len(erange)))
params['C'] = np.zeros((len(crange), len(erange)))

mp_params = np.array([[(i, j, C, e) for j, e in enumerate(erange)] 
                      for i, C in enumerate(crange)]).reshape(-1, 4)

print('Iterations to attempt: %d'%len(mp_params))

Run SVR, collect output iterated over hyperparameters (C, epsilon)<br>
Specify the cost function/loss function used, tolerances, kernel, error metric, etc

In [ ]:
# Paralellize here
# mp_returns = [SVR_mp(_param) for _param in mp_params[:10]]

with mp.get_context('fork').Pool(64) as p:
    mp_returns = p.map(SVR_mp, mp_params, chunksize=1)
    p.close()
    p.join()

for item in mp_returns:
    
    i, j, C, e, r2, mae, mse, mare, model = item
    i, j = int(i), int(j)
        
    params['r2'][i, j] = r2
    params['mse'][i, j] = mse
    params['mae'][i, j] = mae
    params['mare'][i, j] = mare
    params['model'][i, j] = model
    params['epsilon'][i, j] = e
    params['C'][i, j] = C

Plot model performance over time, cost/loss function evolution and skill

In [ ]:
X_train_norm

Plot SVR output with hyperparameters (C, epsilon)<br>
Apply a grid-search method to select best performing parameters

In [ ]:
min_on, indexer, _ = 'R2', np.where(params['r2'] == params['r2'].max()), params['r2'].max()
min_on, indexer, _ = 'MAE', np.where(params['mae'] == params['mae'].min()), params['mae'].min()
min_on, indexer, _ = 'MSE', np.where(params['mse'] == params['mse'].min()), params['mse'].min()
min_on, indexer, _ = 'MARE', np.where(params['mare'] == params['mare'].min()), params['mare'].min()

for min_on in ['r2']:
    
    if min_on in ['mse', 'mae', 'mare']:
        min_max = 'Minimized'
        indexer = np.where(params[min_on] == params[min_on].min())
    elif min_on in ['r2']:
        min_max = 'Maximized'
        indexer = np.where(params[min_on] == params[min_on].max())

    r, c = indexer
    r, c = r[0], c[0]
    r, c, _

    model = params['model'][r, c]
    test_predictions = model.predict(X_test_norm)
    
    y_true = y_test
    y_pred = test_predictions
    print('MARE ', MARE(y_true, y_pred))

    fig, axs = plt.subplots(2, 3, facecolor='w', figsize=(24, 14))
    axs = axs.flatten()

    ax = axs[0]
    cbar = ax.pcolormesh(erange, crange, params['mae'])
    plt.colorbar(cbar, label='mae', ax=ax)
    ax.set_title('Min MAE: %.3f'%params['mae'][r, c])
    ax.scatter(params['epsilon'][r, c], params['C'][r, c], s=500, c='w', marker='+')

    ax = axs[1]
    cbar = ax.pcolormesh(erange, crange, params['mse'])
    plt.colorbar(cbar, label='mse', ax=ax)
    ax.set_title('Min MSE: %.3f'%params['mse'][r, c])
    ax.scatter(params['epsilon'][r, c], params['C'][r, c], s=500, c='w', marker='+')

    ax = axs[2]
    cbar = ax.pcolormesh(erange, crange, params['r2'])
    plt.colorbar(cbar, label='r2', ax=ax)
    ax.set_title('Max R^2: %.3f'%params['r2'][r, c])
    ax.scatter(params['epsilon'][r, c], params['C'][r, c], s=500, c='k', marker='+')

    for ax in axs[:3]:
        ax.set_xlabel('epsilon')
        ax.set_ylabel('C_val')
        ax.set_ylim([crange.min(), crange.max()])
        ax.set_xlim([erange.min(), erange.max()])

    ax = axs[3]
    maxslr = y_test.max() if y_test.max() > y_train.max() else y_train.max()

    ax.hist(y_train, bins=np.arange(0, maxslr, 2), color='g', edgecolor='k', alpha=1.0, label='Train SLR\nn=%d'%len(y_train))
    ax.hist(y_test, bins=np.arange(0, maxslr, 2), color='C0', edgecolor='k', alpha=1.0, label='Test SLR\nn=%d'%len(y_test))
    ax.legend()

    ax.set_xticks(np.arange(0, maxslr+1, 5))
    ax.set_xticklabels(np.arange(0, maxslr+1, 5).astype(int))
    ax.grid()

    ax = axs[4]
    maxslr = test_predictions.max() if test_predictions.max() > y_test.max() else y_test.max()
    maxslr += 5
    ax.scatter(y_test, test_predictions, c='k', s=50, marker='+', linewidth=0.75)
    ax.set_xlabel('Observed SLR')
    ax.set_ylabel('Predicted SLR')
    ax.plot([0, maxslr], [0, maxslr])
    ax.set_xlim([0, maxslr])
    ax.set_ylim([0, maxslr])
    ax.set_aspect('equal')
    ax.grid()

    ax = axs[5]
    error = test_predictions - y_test
    ax.hist(error, bins=np.arange(-30, 30, 2), edgecolor='k')
    ax.set_xlabel('Prediction Error')
    ax.set_ylabel('Count')
    ax.grid()

    plt.suptitle('Support Vector Regression Model\n%s\n%s on: %s\nepsilon %.3f\nc_val: %.3f'%(site_list, min_max, min_on.upper(), params['epsilon'][r, c], params['C'][r, c]))
    plt.show()

### Describe final trained SVR model
Descriptive plots of predictor coefficient rank/influence<br>
Print out model parameters, error metrics, skill scores

In [ ]:
# ranksort = np.argsort(abs(model.coef_))[::-1]
# svr_coefs = model.coef_[0][ranksort][0]
# svr_keys = X_train_norm.keys()[ranksort][0]
# mask = svr_coefs != 0

# fig, ax = plt.subplots(1, 1, facecolor='w', figsize=(8, 14))

# ax.axvline(0, color='k', linewidth=3, zorder=11)
# ax.barh(svr_keys[mask], svr_coefs[mask], color='green', zorder=10, height=0.97)

# for i, k in enumerate(svr_keys):
#     if svr_coefs[i] != 0:
#         ax.text(svr_coefs[i]/2, k, k, zorder=20)

# # ax.invert_yaxis()
# ax.axes.get_yaxis().set_visible(False)
# ax.set_title('SVR Coefs')
# ax.set_xlabel('SVR Coef Value')
# ax.grid(zorder=-1)

# plt.show()

### Save out a usable model along with the input parameters for application
Save with a descriptive filename and a metadata text file!<br>
This will make swapping out models for evaluation much simpler